In [5]:
suits = ["Clubs", "Diamonds", "Hearts", "spades"]
values = list(range(2,11)) + ["Jack", "Queen", "King", "Ace"]

def make_deck(): #creates deck
  deck=list()
  for suit in suits:
    for value in values:
      deck.append((suit, value)) #appends each suit with value
  return deck

In [6]:
class base: #class to control hwo much text is comingout
  SILENT=6
  DEBUG=1
  INFO=2
  WARNING=3
  ERROR=4
  CRITICAL=5

  def __init__(self,level=0):
    self.level=level

  def message(self, level, *args):
    if level >= self.level:
      print(*args)

In [7]:
class Card(base):
  _suits= ["Clubs", "Diamonds", "Hearts", "spades", "shufflecard"]
  _values=list(range(2,11)) + ["Jack", "Queen", "King", "Ace"]

  def init(self, suit, value=None):
    base.init(self)
    self._suit = suit if suit in self._suits else None
    self._values = value if value in self._values else None

    if self._suit is None:
      self.message(self.ERROR, "Error, bad suit", suit)

    if self._value is None and self._suit != "shufflecard":
      self.message(self.ERROR, "Error, bad value", value)

  def value(self):
    return self._value
  def suit(self):
    return self._suit

  def numerical_value(self):
    if self._value == "Ace":
      return 1
    elif self._value in ["Jack", "Queen", "King"]:
      return 10
    else:
      return self._value

  def shuffle_card(self):
    return self._suit == "shufflecard"

  def _str_(self):
    if self.shuffle_card():
      return "shuffle card"
    else:
      return str(self._value) +"of"+self._suit

  _repr_=_str_

In [8]:
import random

class Deck(base):
  _suits = ["Clubs", "Diamonds", "Hearts", "spades", "shufflecard"]
  _values=list(range(2,11)) + ["Jack", "Queen", "King", "Ace"]

  def init(self, n_decks=6):
    base._init(self)
    self._n_decks=n_decks

    self._cards = list()
    for _ in range(self._N_decks):
      self._cards.extend(self._make_deck())

    self._cards.append(Card("ShuffleCard"))

  def _make_deck(self):
    deck=list()
    for suit in self._suits:
      for value in self._values:
        deck.append(Card(suit,value))
    return deck

  def shuffle(self):
    random.shuffle(self._cards)
  def deal(self):
    if len(self._cards)>0:
      return self._cards.pop()
    else:
      for _ in range(self._n_decks):
        self._cards.extend(self._make_deck())
      self.shuffle()
      return self._cards.pop()

In [9]:
my_deck = Deck()
my_deck.shuffle()

[my_deck.deal() for _ in range(10)]

AttributeError: 'Deck' object has no attribute '_cards'

In [11]:
def calc_hand0(hand):
  return sum(map(lambda card: card.numerical_value(),hand))

In [12]:
def calc_hand(hand):
  card_values = list(map(lambda card: card.numerical_value(), hand))

  n_As=len(list(filter(lambda x: x==1, card_values)))

  hand_value = sum(card_values)

  if n_As==0:
    return hand_value

  Ace1=hand_value
  Ace11=hand_value+10

  if Ace11<=21:
    return Ace11
  else:
    return Ace1

In [13]:
calc_hand([Card("Clubs",10), Card("Clubs", 2) Card("Hearts", "Ace")])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-13-b168a41f6447>, line 1)

In [15]:
class PlayerBase(base):
  def init(self, name, n_chips):
    base.init(self)
    self._name = name
    self._n_chips=n_chips

  def name(self):
      return self._name

  def chips(self):
      return self._n_chips

  def pay(self, value=2):
      self._n_chips += value

  def deduct(self, value=2):
      self._n_chips -= value

  def play_hand(self, down_card, upcards, seen_cards):
    raise NotImplementedError


  def __str__(self):
      return self._name + "("+str(self._n_chips)+")"

  __repr__ = __str__

class DealerPlayer(PlayerBase):
  def init(self,threshold=16):
    self._threshold = threshold
    PlayerBase.init(self,"Mr.Dealer", 1000)

  def play_hand(self, down_card,upcards, seen_cards):
    hand_value = calc_hand([down_card]+up_cards)
    return hand_value < self._threshold

class ConsolePlayer(PlayerBase):
  def play_hand(self, down_card, upcards, seen_cards):
    print("DC", down_card)
    print("UC", up_cards)
    print("SC", seen_cards)
    hit_str= input("Hit(Y/N):")
    return hit_str.upper()=="Y"

class Strategy_1_Player(PlayerBase):
  def play_hand(self, down_card, upcards, seen_cards):
    return True

In [ ]:
class Game(base):
  def init(self,n_decks=6):
    base.init(self,self.INFO)
    self._n_decks=n_decks
    self._players=list()
    self._all_players = list()

    self._shuffle=False

  def players(self):
    return self._players

  def all_players(self):
    return self._all_players

  def add_player(self,player):
    self._players.append(player)
    self._all_players.append(player)

  def dcs(self,deck):
    card=deck.deal()
    if card.shuffle_card():
      shuffle=True
      card = deck.deal()
    return card

  def show_status(self,hands,seen_cards):
    self.message(self.INFO,"===========")
    self.message(self.INFO,"hands", hands)
    self.message(self.INFO,"**************")
    self.message(self.INFO,"Players")
    self.message(self.INFO,"**************")
    for i,player in enumerate(self._all_players):
      self.message(self.INFO, i, ":", player)
    self.message(self.INFO,"------------")

  def play_game(self,n_hands):
    self.add_player(DealerPlayer())

    deck=None
    self._shuffle=False

    for i_hand in range(n_hands):

      self._players= list(filter(lambda player: player.chips()>=2, self._players))
      self.message(self.DEBUG,"n players, n all players", len(self.Players), len(self._all_players))
      self.message(self.DEBUG, "starting hand:", i_hand, "/", n_hands)

      if deck is None or self._shuffle:
        self.message(self.DEBUG, "Creating new deck / shuffling")
        deck.shuffle()
        seen_cards = list()
        self._shuffle=False

      hands = list()

      self.message(self.DEBUG, "dealing cards")
      for player_i, player in enumerate(self._players):
        down_card=self.dcs(deck)
        up_cards=list()
        hands.append((down_card, upcards))
        if player_i < len(self._players)-1:
          seen_cards.append(down_card)

      for (down_card,up_cards) in hands:
        up_cards.append(self.dcs(deck))
        seen_cards.append(up_cards[-1])

      self.show_status(hands,seen_cards)

      for player-i, ((down_cards,player) in enumerate(zip(hands,self._players))):
        self.message(self.DEBUG, "Asking player", player_i, "to play")
        self.message(self.DEBUG, "player", player_i, "hand total:", calc_hand([down_card]+up_cards))
        hit=True
        this_hand_up_cards=list()
        while(hit):
          hit=player.play_hand(down_card,up_cards,seen_cards)

          if hit:
            self.message(self.DEBUG,"player",player_i,"hit")
            card=self.dcs(deck)
            up_cards.append(card)
            this_hand_up_cards.append(card)
            hand_value=calc_hand_value([down_card]+up_cards)
            self.message(self.DEBUG,"hand value:",hand_value)
            if hand_value < 21:
              hit = True
            else:
              hit=False
              if hand_value>21:
                self.message(self.DEBUG,"player", player_i, "Busted")
              else:
                self.message(self.DEBUG,"player",player_i,"Got 21")
            else:
              self.message(self.DEBUG,"player", player-i,"stay")

        seen_cards.append(down_card)
        seen_cards.extend(this_hand_up_cards)

      hand_values=[calc_hand_value([hand[0]]+hand[1]) for hand in hands]
      self.message(self.DEBUG, "hand values:", hand_values)

      if hand_values[-1]==21:
        self.message(self.DEBUG,"Dealer got 21")
        for player_i, (hand_value,player) in enumerate(zip(hand_values[:-1], self._players[:-1])):
          if hand_value==21:
            self.message(self.DEBUG, "player", player_i, "got 21. paying 3 chips")
            player.pay(3)
          else:
            self.message(self.DEBUG, "player", player_i,"busted or dealer won. deducting 2 chips")
      if hand_values[-1]>21:
        self.message(self.DEBUG,"dealer busted")
        for player_i, (hand_value,player) in enumerate(zip(hand_values[:-1], self._players[:-1])):
          if hand_value==21:
            self.message(self.DEBUG,"player", player_i, "got 21.paying 3 chips")
            player.pay(3)
          elif hand_value>21:
            self.message(self.DEBUG,"player", player_i, "busted. deducting 2 chips")
          else:
            self.message(self.DEBUG,"player", player_i, "paying 2 chips")

      if hand_values[-1]<21:
        self.message(self.DEBUG, "dealer hand is:", hand_values[-1])
        for player_i, (hand_value,player) in enumerate(zip(hand_values[:-1], self._players[:-1])):
          if hand_value==21:
            self.message(self.DEBUG, "player", player_i, "got 21. paying 3 chips")
            player.pay(3)
          elif hand_value>21:
            self.message(self.DEBUG, "player", player_i, "busted. deducting 2 chips")
            player.deduct(2)




In [ ]:
my_game=Game()
my_game.level=Game.DEBUG

my_game.add_player(Strategy_1_Player("Dummy player", 100))
my_game.add_player(ConsolePlayer("ME!", 100))

my_game.play_game(2)

In [ ]:
class Strategy_1_Player(PlayerBase):
  def play_hand(self, down_card, up_cards, seen_cards):
    return True

class Strategy_2_Player(PlayerBase):
  def play_hand(self, down_card, up_cards, seen_cards):
    """
    """
    return True
class DealerLikePlayer(PlayerBase):
  def init(self, name, chips, threshold=16):
    self.threshold = threshold
    PlayerBase.init(self,name,chips)
  def play_hand(self, down_card, up_cards, seen_cards):
    hand_value < self._threshold

In [ ]:
my_game=Game()
my_game.level=Game.SILENT

my_game.add_player(Strategy_1_Player("Dummy player", 100))
my_game.add_player(DealerLikePlayer("stays on 13", 100,13))
my_game.add_player(DealerLikePlayer("stays on 14", 100,14))
my_game.add_player(DealerLikePlayer("stays on 15", 100,15))
my_game.add_player(DealerLikePlayer("stays on 16", 100,16))
my_game.add_player(DealerLikePlayer("stays on 17", 100,17))
my_game.add_player(DealerLikePlayer("stays on 18", 100,18))
my_game.add_player(DealerLikePlayer("stays on 19", 100,19))
my_game.add_player(DealerLikePlayer("stays on 20", 100,20))

my_glame.play_game(10)

In [ ]:
[player.chips() for player in my_game.players()]

In [ ]:
def play_a_round(n_hands=50):
  my_game=Game()
  my_game.level=Game.ERROR

  my_game.add_player(Strategy_1_Player("Dummy player",100))
  my_game.add_player(DealerLikePlayer("stays on 13", 100,13))
  my_game.add_player(DealerLikePlayer("stays on 14", 100,14))
  my_game.add_player(DealerLikePlayer("stays on 15", 100,15))
  my_game.add_player(DealerLikePlayer("stays on 16", 100,16))
  my_game.add_player(DealerLikePlayer("stays on 17", 100,17))
  my_game.add_player(DealerLikePlayer("stays on 18", 100,18))
  my_game.add_player(DealerLikePlayer("stays on 19", 100,19))
  my_game.add_player(DealerLikePlayer("stays on 20", 100,20))

  my_glame.play_game(_n_hands)
  return [player.chips() for player in my_game.all_players()]

In [ ]:
results=play_a_round(100)

In [ ]:
results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
res = np.array(results)
res = res [:,:-1]

In [ ]:
for i in range(res.shape[1]):
  plt.hist(res[:,i], label="player"+str(i),alpha=.5)
plt.legen()
plt.show


In [ ]:
m=np.mean(res, axis=0)
s=np.std(res, axis=0)
plt.errorbar(range(len(m),m,s)